In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.datafactory import DataFactoryManagementClient
from azure.mgmt.datafactory.models import *

subscription_id = 'your_subscription_id'
resource_group_name = 'your_resource_group_name'
adf_name = 'your_data_factory_name'
location = 'your_data_factory_location'

credential = DefaultAzureCredential()
adf_client = DataFactoryManagementClient(credential, subscription_id)

oracle_linked_service = LinkedServiceResource(properties={
    'type': 'Oracle',
    'typeProperties': {
        'connectionString': 'your_oracle_connection_string'
    }
})

salesforce_linked_service = LinkedServiceResource(properties={
    'type': 'Salesforce',
    'typeProperties': {
        'username': 'your_salesforce_username',
        'password': 'your_salesforce_password',
        'securityToken': 'your_salesforce_security_token'
    }
})

ftp_linked_service = LinkedServiceResource(properties={
    'type': 'FTP',
    'typeProperties': {
        'host': 'ftp.example.com',
        'username': 'your_ftp_username',
        'password': 'your_ftp_password'
    }
})

blob_storage_linked_service = LinkedServiceResource(properties={
    'type': 'AzureBlobStorage',
    'typeProperties': {
        'connectionString': 'your_blob_storage_connection_string'
    }
})

sql_db_linked_service = LinkedServiceResource(properties={
    'type': 'AzureSqlDatabase',
    'typeProperties': {
        'connectionString': 'your_sql_db_connection_string'
    }
})

oracle_dataset = DatasetResource(properties={
    'linkedServiceName': {
        'referenceName': 'OracleLinkedService',
        'type': 'LinkedServiceReference'
    },
    'type': 'OracleTable',
    'schema': []
})

salesforce_dataset = DatasetResource(properties={
    'linkedServiceName': {
        'referenceName': 'SalesforceLinkedService',
        'type': 'LinkedServiceReference'
    },
    'type': 'SalesforceObject',
    'schema': []
})

ftp_dataset = DatasetResource(properties={
    'linkedServiceName': {
        'referenceName': 'FTPLinkedService',
        'type': 'LinkedServiceReference'
    },
    'type': 'File',
    'typeProperties': {
        'folderPath': 'ftp_folder_path',
        'fileName': '*.csv'
    },
    'schema': []
})

blob_dataset = DatasetResource(properties={
    'linkedServiceName': {
        'referenceName': 'AzureBlobStorageLinkedService',
        'type': 'LinkedServiceReference'
    },
    'type': 'AzureBlob',
    'typeProperties': {
        'folderPath': 'container_name/folder_path',
        'fileName': '*.csv'
    },
    'schema': []
})

sql_dataset = DatasetResource(properties={
    'linkedServiceName': {
        'referenceName': 'AzureSqlDatabaseLinkedService',
        'type': 'LinkedServiceReference'
    },
    'type': 'AzureSqlTable',
    'typeProperties': {
        'tableName': 'target_table_name'
    },
    'schema': []
})

oracle_pipeline = PipelineResource(properties={
    'activities': [{
        'name': 'CopyFromOracle',
        'type': 'Copy',
        'inputs': [{
            'referenceName': 'OracleDataset',
            'type': 'DatasetReference'
        }],
        'outputs': [{
            'referenceName': 'SQLDataset',
            'type': 'DatasetReference'
        }],
        'typeProperties': {
            'source': {
                'type': 'OracleSource',
                'query': 'SELECT * FROM table_name WHERE update_date >= @pipeline().parameters.windowStart'
            },
            'sink': {
                'type': 'SqlSink'
            }
        }
    }],
    'parameters': [{
        'name': 'windowStart',
        'type': 'String',
        'defaultValue': '2024-01-01T00:00:00Z'
    }]
})

salesforce_pipeline = PipelineResource(properties={
    'activities': [{
        'name': 'CopyFromSalesforce',
        'type': 'Copy',
        'inputs': [{
            'referenceName': 'SalesforceDataset',
            'type': 'DatasetReference'
        }],
        'outputs': [{
            'referenceName': 'SQLDataset',
            'type': 'DatasetReference'
        }],
        'typeProperties': {
            'source': {
                'type': 'SalesforceSource',
                'query': 'SELECT Id, Name, Email FROM Account WHERE SystemModstamp >= @pipeline().parameters.windowStart'
            },
            'sink': {
                'type': 'SqlSink'
            }
        }
    }],
    'parameters': [{
        'name': 'windowStart',
        'type': 'String',
        'defaultValue': '2024-01-01T00:00:00Z'
    }]
})

ftp_pipeline = PipelineResource(properties={
    'activities': [{
        'name': 'CopyFromFTPToBlob',
        'type': 'Copy',
        'inputs': [{
            'referenceName': 'FTPDataset',
            'type': 'DatasetReference'
        }],
        'outputs': [{
            'referenceName': 'BlobDataset',
            'type': 'DatasetReference'
        }],
        'typeProperties': {
            'source': {
                'type': 'FileSource'
            },
            'sink': {
                'type': 'BlobSink'
            }
        }
    }]
})

adf_client.linked_services.create_or_update(resource_group_name, adf_name, 'OracleLinkedService', oracle_linked_service)
adf_client.linked_services.create_or_update(resource_group_name, adf_name, 'SalesforceLinkedService', salesforce_linked_service)
adf_client.linked_services.create_or_update(resource_group_name, adf_name, 'FTPLinkedService', ftp_linked_service)
adf_client.linked_services.create_or_update(resource_group_name, adf_name, 'AzureBlobStorageLinkedService', blob_storage_linked_service)
adf_client.linked_services.create_or_update(resource_group_name, adf_name, 'AzureSqlDatabaseLinkedService', sql_db_linked_service)

adf_client.datasets.create_or_update(resource_group_name, adf_name, 'OracleDataset', oracle_dataset)
adf_client.datasets.create_or_update(resource_group_name, adf_name, 'SalesforceDataset', salesforce_dataset)
adf_client.datasets.create_or_update(resource_group_name, adf_name, 'FTPDataset', ftp_dataset)
adf_client.datasets.create_or_update(resource_group_name, adf_name, 'BlobDataset', blob_dataset)
adf_client.datasets.create_or_update(resource_group_name, adf_name, 'SQLDataset', sql_dataset)

adf_client.pipelines.create_or_update(resource_group_name, adf_name, 'OracleToSQLPipeline', oracle_pipeline)
adf_client.pipelines.create_or_update(resource_group_name, adf_name, 'SalesforceToSQLPipeline', salesforce_pipeline)
adf_client.pipelines.create_or_update(resource_group_name, adf_name, 'FTPToBlobPipeline', ftp_pipeline)

print('Azure Data Factory pipelines created successfully.')
